In [1]:
from datetime import datetime
import requests
import json

In [3]:
class User:
    def __init__(self, data):
        self.data = data
    
    def get_name(self):
        return self.data['name']
    
    def get_pis(self):
        return self.data['pis']
        

In [4]:
class AFD:
    def __init__(self, afd, session):
        self.afd = afd;
        self.session = session;
        
    def get_original(self):
        return self.afd

    def get_parsed(self):
        string_date = self.afd[10:18]
        string_time = self.afd[18:22]
        pis = int(self.afd[22:34])
        fmt_string_date = f"{string_date[0:2]}/{string_date[2:4]}/{string_date[4:8]} {string_time[0:2]}:{string_time[2:5]}"

        return {
        "nsr" : self.afd[0:9],
        "tipo_registro" : self.afd[9],
        "pis" : int(self.afd[22:34]),
        "data" : fmt_string_date,
        }
        

In [9]:
class Session:
    options = {
        'headers': {
            'Content-type': 'application/json',
            'Accept': 'text/plain',
        },
        'verify':False,
    }
    
    def __init__(self, user, password, host):
        self.host = host
        data = {
            "login": user,
            "password": password
        }
        response = requests.post(f"{host}/login.fcgi", data=json.dumps(data), **self.options)
        
        self.session = response.json()['session']
    
    def post(self, url ,data):
        return requests.post(f"{self.host}/{url}.fcgi", data=json.dumps(data), **self.options, params={"session": self.session})
    
    def get_user_pis(self, list_pis):
        data = {
            "users_pis": [list_pis]
        }
        response = self.post("load_users", data)

        return_data = response.json()['users']
        
        return return_data
    
    
    def get_afds(self, day, month, year, nsr):
        data = {
            "initial_date": {
                "day": day,
                "month": month,
                "year": year,
                "initial_nsr": nsr
            }
        }
        
        response = self.post("get_afd", data)
        txt = response.text.split('\r\n')
        return_data = txt
        
        return return_data
    

In [10]:
def get_afds(session, day, month, year):
    afds_txt = session.get_afds(day, month ,year, 1)
    afds = []
    
    for afd in afds_txt:
        try:
            afds.append(AFD(afd, session))
            print(AFD(afd, session).get_parsed())
            
        except ValueError:
            continue

    return afds


def get_users_from_afds(session, afds):
    list_pis = map(lambda x: x['pis'], afds)
    list_pis = set(list_pis)
    
    return session.get_user_pis(list_pis)
    

In [8]:
session = Session("admin", "admin", "https://192.168.0.98")
afds = get_afds(session)



/home/usuario/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.98'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/usuario/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.98'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'nsr': '000017307', 'tipo_registro': '3', 'pis': 13006643193, 'data': '24/07/2020 08:23'}
{'nsr': '000017308', 'tipo_registro': '3', 'pis': 13345907193, 'data': '24/07/2020 08:46'}
{'nsr': '000017309', 'tipo_registro': '3', 'pis': 20627430850, 'data': '24/07/2020 08:46'}
{'nsr': '000017311', 'tipo_registro': '3', 'pis': 16666276621, 'data': '31/07/2020 15:43'}
{'nsr': '000017312', 'tipo_registro': '3', 'pis': 12885302072, 'data': '01/08/2020 07:29'}
{'nsr': '000017313', 'tipo_registro': '3', 'pis': 16160707087, 'data': '01/08/2020 07:29'}
{'nsr': '000017314', 'tipo_registro': '3', 'pis': 16037269654, 'data': '01/08/2020 07:29'}
{'nsr': '000017315', 'tipo_registro': '3', 'pis': 20627430850, 'data': '01/08/2020 07:29'}
{'nsr': '000017316', 'tipo_registro': '3', 'pis': 16012620889, 'data': '01/08/2020 07:30'}
{'nsr': '000017317', 'tipo_registro': '3', 'pis': 16186774261, 'data': '01/08/2020 07:34'}
{'nsr': '000017318', 'tipo_registro': '3', 'pis': 16037269654, 'data': '01/08/2020 12:15'}

{'nsr': '000019229', 'tipo_registro': '3', 'pis': 14103407197, 'data': '25/09/2020 07:32'}
{'nsr': '000019230', 'tipo_registro': '3', 'pis': 16037269654, 'data': '25/09/2020 07:33'}
{'nsr': '000019231', 'tipo_registro': '3', 'pis': 13630906191, 'data': '25/09/2020 07:33'}
{'nsr': '000019232', 'tipo_registro': '3', 'pis': 13006643193, 'data': '25/09/2020 07:34'}
{'nsr': '000019233', 'tipo_registro': '3', 'pis': 16186774261, 'data': '25/09/2020 07:39'}
{'nsr': '000019234', 'tipo_registro': '3', 'pis': 14428809006, 'data': '25/09/2020 08:00'}
{'nsr': '000019235', 'tipo_registro': '3', 'pis': 16074142174, 'data': '25/09/2020 08:04'}
{'nsr': '000019236', 'tipo_registro': '3', 'pis': 20113646709, 'data': '25/09/2020 08:18'}
{'nsr': '000019237', 'tipo_registro': '3', 'pis': 13630906191, 'data': '25/09/2020 11:08'}
{'nsr': '000019238', 'tipo_registro': '3', 'pis': 14103407197, 'data': '25/09/2020 11:13'}
{'nsr': '000019239', 'tipo_registro': '3', 'pis': 16037269654, 'data': '25/09/2020 11:18'}